# Generating sequences from model
Code samples 10 times from the first protein in the dataset (WT). Note that the latent vectors of each sequence have already been calculated beforehand, using save_latent_from_config() method in examine_model.py
# TODO
- Replace pytorch tensors with pandas dataframes. Or possibly np.array()?

In [1]:
# libraries
import pickle
import torch
import numpy as np

# local files
from read_config import Config
import utils

ModuleNotFoundError: No module named 'torch'

In [ ]:
config_path = '../config_1000_15.yaml'
config = Config(config_path)
model = utils.load_model_from_config(config)

MSA = utils.get_dataset_from_config(config)
# dimensions of latent_vecs: seq x (means, log_variances)
latent_vecs = pickle.load(open(config.latent_fullpath, 'rb'))['latent']
latent_means = [row[0] for row in latent_vecs]
latent_log_vars = [row[1] for row in latent_vecs]

In [ ]:
means = latent_means[0]
log_vars = latent_log_vars[0]
# means = torch.tensor([0.0]*15)
# log_vars = torch.tensor([0.0]*15)
no_var = torch.tensor([-np.inf]*15)

sample_size = 10000


outputs = []
with torch.no_grad():
    for i in range(sample_size):
        if i==0:
            encoding = model.reparameterize(means, no_var)
        else:
            encoding = model.reparameterize(means, log_vars)
        output = model.decoder(encoding)
        outputs.append(output)
# converst list of 1D tensors into a single 2D tensor
outputs = torch.stack(outputs)
one_hots = utils.softmax(outputs)
# one_hots dimensions before: sample_size x onehotencoding
one_hots = one_hots.reshape(sample_size, -1, 21)
# onehots dimensions after: sample_size x seq_length x 21_amino_acids

In [ ]:
from Bio import SeqIO as seq

aligned = np.array(utils.one_hots_to_proteins(one_hots))

seq.write(aligned, "wt_samples.fasta", "fasta")

# seqs = np.array([utils.remove_gaps(seq) for seq in aligned])

# for seq in seqs:
#     print(len(seq))
# for seq in seqs:
#     print(''.join(seq))